In [116]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re


# Missing data GDSC

In [153]:
gdsc_dr = pd.read_csv('data/GDSC/metrics.csv').rename(columns={'Cell_line':'CCL'})
gdsc_ge = pd.read_csv('data/GDSC/ge.csv', sep='\t').rename(columns={'Unnamed: 0':'CCL'}).transpose()
new = pd.read_csv('data/GDSC/new.csv').rename(columns={'CELL_LINE_NAME':'CCL'})
new['CCL'] = new['CCL'].str.replace('-', '').str.replace(' ','').str.upper()

In [154]:
print(str(len(gdsc_dr['CCL'].unique())) + ' different CCLs in Pozdeyev\'s data')

707 different CCLs in Pozdeyev's data


In [155]:
print(str(len(gdsc_dr['Drug_name'].unique())) + ' different Drugs in Pozdeyev\'s data')

139 different Drugs in Pozdeyev's data


In [156]:
print(str(len(new['CCL'].unique())) + ' different CCLs in new data')

808 different CCLs in new data


In [157]:
print(str(len(new['DRUG_ID'].unique())) + ' different Drugs in new data')

200 different Drugs in new data


### Deletes the symbols from CCL ID to normalize format

In [158]:
def clean(s):
    new = []
    for i in s:
        new.append(re.sub("[^A-Za-z0-9]", '', i).upper())
    return new

In [159]:
poz_r = list(gdsc_dr['CCL'].unique())
cell_r = list(gdsc_ge.index.to_series().reset_index()[0])
gdsc_r = list(pd.Series(new['CCL'].unique()))

cell = clean(cell_r)
poz = clean(poz_r)
gdsc = clean(gdsc_r)

Pozdeyev's CCLs missing from CellMinerCDB

In [160]:
miss1 = []
for ccl in poz:
    if ccl not in cell:
        miss1.append(ccl)
len(miss1)

8

New data CCLs missing from CellMinerCDB

In [161]:
miss2 = []
for ccl in gdsc:
    if ccl not in cell:
        miss2.append(ccl)
len(miss2)

19

In [162]:
gdsc2 = {k:re.sub("[^A-Za-z0-9]", '', k).upper() for k in gdsc_r if re.sub("[^A-Za-z0-9]", '', k).upper() not in miss2}

These rules are made by looking at the 'missing' CCLs that turned out to simply have different names

In [163]:
for i in miss2:
    if i[0:3]=='NCI':
        gdsc2[i] = i.lstrip('NCI')
    elif i[0:3]=='SNU':
        gdsc2[i] = 'NCI' + i


In [164]:
gdsc2['NTERA2CLD1'] = ('NTERASCLD1')

In [165]:
new['CCL'] = new['CCL'].replace(gdsc2)

Same is done now for Pozdayev's data.

## One question, I turn TTTHYROID into TT because of [this](https://web.expasy.org/cellosaurus/CVCL_1774). Not quite sure, let's maybe discuss it.

In [130]:
poz2 = {k:re.sub("[^A-Za-z0-9]", '', k) for k in poz_r if re.sub("[^A-Za-z0-9]", '', k).upper() not in miss1}

In [131]:
missv = ['7860', 'COLO320HSR','KNS81FD', 'EOL1CELL', 'NCIH510A','TT' , 'U266', 'U031']
for i, ele in enumerate(miss1):
    poz2[ele] = missv[i]

In [132]:
gdsc_dr['CCL'] = gdsc_dr['CCL'].replace(poz2)

In [133]:
gdsc_dr.to_csv('data/Processed/gdsc_poz_dr.csv', index =False)

In [134]:
gdsc_ge = gdsc_ge.reset_index()
gdsc_ge.columns = gdsc_ge.iloc[0]
gdsc_ge = gdsc_ge[1:]
cellrep = {k:v for k in cell v in }

In [152]:
gdsc_ge

,CCL,A1BG,A1CF,A2M,A2ML1,A3GALT2,A3GALT2P,A4GALT,A4GNT,AAAS,...,ZW10,ZWILCH,ZWINT,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
1,201T,3.17242,2.93284,3.39045,2.71703,2.67498,2.67498,3.48303,3.1235,4.75917,...,7.26434,6.20224,6.28252,4.04948,4.93531,2.74394,4.77964,4.70476,4.6623,8.80625
2,22RV1,3.53794,6.36465,5.33244,2.92303,2.81538,2.81538,3.24112,3.26263,4.72216,...,8.37838,7.52632,9.27417,3.78576,5.20193,2.74257,4.63502,3.94019,4.65866,8.08867
3,23132-87,3.37095,6.28488,3.48568,2.83156,2.91337,2.91337,3.31303,3.09653,4.87362,...,8.58629,6.52671,8.02195,3.52058,4.95737,2.63693,4.35012,4.39581,4.59858,7.95115
4,42-MG-BA,6.01674,3.19189,3.24948,2.80896,2.6262,2.6262,3.06339,3.35685,4.91996,...,8.57463,7.27485,7.68393,3.47306,5.23711,2.75974,5.51799,4.39196,4.15771,8.94488
5,451Lu,5.72524,2.97195,10.7685,2.92855,2.75028,2.75028,3.36712,3.16265,5.14418,...,7.80586,7.94224,9.38534,3.29279,5.53094,2.58216,5.08715,5.19761,4.92217,7.4706
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1076,YH-13,5.16786,3.07284,3.04907,3.09807,3.02972,3.02972,2.99009,3.31857,4.56187,...,8.47652,8.04813,8.2353,3.3059,4.74561,2.57763,5.21584,7.00354,4.20382,7.91928
1077,YKG-1,4.71162,3.06794,3.30693,3.03796,2.71344,2.71344,3.1544,2.98892,4.58681,...,6.98386,7.97826,9.15039,3.97241,5.16608,2.81345,5.11719,5.08564,4.91817,8.20834
1078,YMB-1-E,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1079,YT,3.14359,3.77979,5.09516,2.87933,2.74909,2.74909,3.18623,3.35959,4.51172,...,8.46731,8.19677,9.20568,3.51532,5.15234,2.47275,5.04678,6.01671,5.16818,7.06395


In [135]:
gdsc_ge.to_csv('data/Processed/gdsc_cell_ge.csv', index=False)

707

# Missing CCLE data

In [136]:
ccle_ge = pd.read_csv('data/CCLE/ge.csv', sep='\t').rename(columns={'Unnamed: 0':'CCL'}).set_index('CCL').transpose()
ccle_dr = pd.read_csv('data/CCLE/metrics.csv').rename(columns={'Cell_line':'CCL'})

In [137]:
poz = list(ccle_dr['CCL'].unique())
cell = list(ccle_ge.index.to_series().str.upper().reset_index()[0])

ncell = []
for i in cell:
    ncell.append(re.sub("[^A-Za-z0-9]", '', i))
cell = ncell

In [138]:
len(poz)

504

In [139]:
miss = []
for ccl in poz:
    if ccl not in cell2:
        miss.append(ccl)
miss

['BGC823',
 'GLC82',
 'KP1N',
 'KP1NL',
 'M059J',
 'MB157',
 'OVCAR3',
 'SF8657',
 'SNUC2B']

### Now I substitute by the correct names

### I look for the following 9 cell lines [here](https://www.expasy.org/) and find what the problem with them is.

BGC823, GLC82, KP1N and KP1NL are problematic (shown to be derivatives of HeLa and PANC-1) cell lines so I will drop them.

MB157 and OVCAR3 are under different names, namely MDAMB157, NIH:OVCAR3

_M059J_ wasn't found on cellminercdb, there is a M059K but idk if it's the same, maybe download M059J from [here](https://web.expasy.org/cellosaurus/CVCL_0400). It is on GDSC's cellminercdb dataset.

_SF8657_ wasn't found,  can be downloaded from [here] (https://web.expasy.org/cellosaurus/CVCL_K085)

_SNUC2B_ can't be found on this cellminercdb dataset but it is on GDSC's, here you find it under the alias SNUC2A according [to] (https://web.expasy.org/cellosaurus/CVCL_1710)

In [147]:
d = ccle_dr[~ccle_dr['CCL'].isin(['BGC823', 'GLC82', 'KP1N', 'KP1NL', 'M059J', 'SF8657', 'SNUC2B'])]

MB157
OVCAR3


# Missing CTRP data

In [459]:
ctrp_ge = pd.read_csv('data/CTRP/cell_ge.csv', sep='\t').rename(columns={'Unnamed: 0':'CCL'}).set_index('CCL').transpose()
ctrp_dr = pd.read_csv('data/CTRP/metrics.csv').rename(columns={'Cell_line':'CCL'})

In [451]:
import re
poz = list(ctrp_dr['CCL'].unique())
cell3 = list(ctrp_ge.index.to_series().str.upper().reset_index()[0])

ncell = []
for i in cell3:
    ncell.append(re.sub("[^A-Za-z0-9]", '', i))
cell3 = ncell

In [453]:
miss = []
for ccl in poz:
    if ccl not in cell3:
        miss.append(ccl)
len(miss)

66

In [414]:
miss

['2004',
 'ASKA',
 'BT112',
 'BT131',
 'BT139',
 'BT145',
 'BT147',
 'BT159',
 'BT16',
 'BT164',
 'BT172',
 'BT179',
 'BT216',
 'BT224',
 'BT228',
 'BT231',
 'BT232',
 'BT239',
 'BT245',
 'BT248',
 'BT271',
 'BT286',
 'BT320',
 'BT328',
 'BT330',
 'BT333',
 'BT340',
 'BT359',
 'BT416',
 'BT422',
 'BT428',
 'BT440',
 'BT444',
 'BT482',
 'BT498',
 'BT504',
 'H292',
 'HS578BST',
 'HS888LU',
 'HSTS',
 'KRIJ',
 'PSTS',
 'SW982',
 'SYO1',
 'TM8716',
 'TTTHYROID',
 'UCH1',
 'YAMATO',
 'BT187']

In [415]:
cell3

['CHP126',
 'CHP212',
 'IMR32',
 'KELLY',
 'KPNSI9S',
 'KPNYN',
 'MHHNB11',
 'NB1',
 'NH6',
 'SIMA',
 'SKNAS',
 'SKNBE2',
 'SKNDZ',
 'SKNFI',
 'SKNSH',
 'HUCCT1',
 'HUH28',
 'SNU1079',
 'SNU1196',
 'SNU245',
 'SNU308',
 'SNU478',
 'SNU869',
 'A673',
 'CADOES1',
 'CAL78',
 'G292CLONEA141B1',
 'HOS',
 'HS822T',
 'HS888T',
 'MG63',
 'MHHES1',
 'RDES',
 'SAOS2',
 'SJSA1',
 'SKES1',
 'SKNMC',
 'SW1353',
 'TC71',
 'U2OS',
 'AU565',
 'BT20',
 'BT474',
 'BT549',
 'CAL120',
 'CAL148',
 'CAL51',
 'CAL851',
 'CAMA1',
 'DU4475',
 'EFM192A',
 'HCC1143',
 'HCC1395',
 'HCC1419',
 'HCC1428',
 'HCC1500',
 'HCC1569',
 'HCC1806',
 'HCC1937',
 'HCC1954',
 'HCC202',
 'HCC2218',
 'HCC38',
 'HDQP1',
 'HMC18',
 'HS578T',
 'JIMT1',
 'KPL1',
 'MCF7',
 'MDAMB157',
 'MDAMB231',
 'MDAMB361',
 'MDAMB415',
 'MDAMB453',
 'MDAMB468',
 'SKBR3',
 'T47D',
 'UACC812',
 'ZR751',
 'ZR7530',
 '42MGBA',
 '8MGBA',
 'A172',
 'AM38',
 'BECKER',
 'CAS1',
 'CCFSTTG1',
 'D283MED',
 'DAOY',
 'DBTRG05MG',
 'DKMG',
 'GAMG',
 'GB1',
 '

## It seems like the Pozdeyev data isn't too good regarding gene expression data, therefore I try to download the ctrp data directly

In [402]:
lo2 = pd.read_csv('data/ctrp2.csv', sep=';')

In [403]:
lo = lo2['ccl_name']

In [404]:
miss = []
for ccl in lo:
    if ccl not in cell2:
        miss.append(ccl)
len(miss)

8

In [411]:
set(miss)

{'1\xa0MKN',
 '45\xa0MKN',
 '74\xa0MKN',
 '7\xa0MCF',
 '7\xa0MKN',
 'H292',
 'TTTHYROID'}

In [409]:
dif = ['MCF7', 'NCIH292', 'MKN74', 'MKN1', 'MKN45', 'MKN7', 'TT']
for i in set(miss):
    

In [438]:
g = pd.read_csv('data/CTRP/ctrp_g.csv', sep='\t')
g

,master_ccl_id,ccl_name,ccle_primary_site,ccle_primary_hist,ccle_hist_subtype_1
0,1,697,haematopoietic_and_lymphoid_tissue,lymphoid_neoplasm,acute_lymphoblastic_B_cell_leukaemia
1,3,5637,urinary_tract,carcinoma,NaN
2,4,2313287,stomach,carcinoma,adenocarcinoma
3,7,22RV1,prostate,carcinoma,NaN
4,8,253J,urinary_tract,carcinoma,transitional_cell_carcinoma
...,...,...,...,...,...
830,155511,NCIH2291,lung,carcinoma,adenocarcinoma
831,155513,NCIH2342,lung,carcinoma,adenocarcinoma
832,155518,NCIH2405,lung,carcinoma,adenocarcinoma
833,155519,NCIH2444,lung,carcinoma,non_small_cell_carcinoma


In [433]:
ge = pd.read_csv('data/CTRP/ctrp_ge.csv', sep='\t')

In [447]:
miss = []
for i in g['ccl_name'].unique():
    if i not in poz:
        miss.append(i)

print(len(miss))

0


In [445]:
len(poz)

887

In [446]:
len(g['ccl_name'].unique())

835